<img src="../media/onda_logo.jpg" alt="logo onda" height="200" width="200">

# AOI search via a Jupyter notebook
This trial notebook allows the geographical search of ONDA EO data offer by simutating a little 🌍 [ONDA Catalogue](https://catalogue.onda-dias.eu/catalogue/) via Jupyter Notebook.

## Send the query: set filters & draw a polygon on the map

- Set products filters (mission, type and sensing range)
- Select an area of interest (AOI) choosing a shape among the buttons on the left side
- Only one polygon per time is supported by this version

⚠️ Do not scroll world window and draw your AOI in a world copy. This will generate non valid coordinates for the query 👉[Issue #96](https://github.com/jupyter-widgets/ipyleaflet/issues/96)<br>
⚠️ Remember to **Submit** each widget field to filter research results to overwrite previous selections saved.<br>
⚠️ Please note that all the previous product lists placed in the *output* folder and named after `product_list*.txt` will be erased, if present.

👉 Click on the **RUN** button and send your query via the [OData API](https://www.onda-dias.eu/cms/knowledge-base/odata-odata-open-data-protocol/) provided interface withing CLEOPE workspace. Wait untill the process is concluded; the overall duration will depend on the query.

In [ ]:
%run -i ../modules/catalogue.py

## Visualise layers on the map and save selections to file

Results are here interactively visualised so that you can choose products depending on the their footprint intersection upon the drawn AOI.<br>
Choose products you wish to collect and submit each of them. This will produce a products list in the *outputs* folder placed in the current working directory. 

In [ ]:
%run -i ../modules/interactive_layers.py

## Work with the product list text file
Product list file named *product_list.txt* contains the name of all the products you decided to save up previously. Now in order to work with this product list you can choose to proceed with one of the following options.

### 1. Use ONDA Advanced API (ENS)
In this case you need to convert your product list into ENS format. Run the following cell to obtain a product list compliant to ENS [ENS path to products](../../userguide.md/#pseudopaths) and then use this list to start your own EO data processing.<br>The input file list can be put as the `file` argument of the `read_product_list` function, while the output filename can be set as `outfile` argument of the `pseudopath` function.

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser("~"),"CLEOPE/Trials/modules"))
import qm

In [ ]:
pseudopaths = qm.pseudopath(dataframe=qm.read_product_list(file="outputs/product_list.txt"),
                            outfile="outputs/product_list_remote.txt")

### 2. Download your custom product list 

In this case you need to download products using the [OData API](https://www.onda-dias.eu/cms/knowledge-base/odata-odata-open-data-protocol/) provided interface. Just provide your own ONDA credentials and the input file list of products (*product_list.txt* file in the example below). The downloaded files will be put in the *local_files* folder in your own workspace and the *list_local.txt* file will be created containing the path to downloaded products.

In [ ]:
username = "username"; password = "password" # ONDA auth required
r = qm.download_list(file='outputs/product_list.txt',username=username,password=password) # download products from your custom list